In [5]:
import requests
import json



import matplotlib.pyplot as plt
import seaborn as sns

from pandasdmx import Request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import geopandas as gpd

import time

import csv

import sys, os
from sqlalchemy import create_engine

import pymssql
import sqlalchemy as db

from functools import reduce

import folium # map rendering library
from sklearn.neighbors import NearestNeighbors



/home/holborn/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


### Here API

In [159]:
API_Key = 't9X9EF_AkG_qVZ8Q4dWSgyJuMhSklDP7i_gqX5EMunQ'
URL = 'https://router.hereapi.com/v8/routes?'
# TRANSPORTMODE ='transportMode=pedestrian' 
PARAMS = '&origin=-37.8046041,144.9454422&destination=-37.8102306,144.9539645&return=polyline,summary&apikey='

TRANSPORTMODE ='pedestrian' 

RETURN = 'polyline'

DESTINATION_X = '-37.8102306'
DESTINATION_Y = '144.9539645'
ORIGIN_X = '-37.8046041'
ORIGIN_Y = '144.9454422'

### Get Travel Time

In [160]:
def get_transport(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://router.hereapi.com/v8/routes?transportMode={}&origin={},{}&destination={},{}&return={},summary&apikey={}'.format(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key)
    items = requests.get(url).json()['routes'][0]['sections']
    transport = json_normalize(items).set_index('id')
    print(url)
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- HERO request for ",nb,"returned no results --")
        return None
    return transport


### FourSquare API

In [161]:
#intialise Foursquare API credentials
url = 'https://api.foursquare.com/v2/venues/explore'


CLIENT_ID = 'ZY2UKTXC3KLI2DSJYFZPC3CPXMU4SGKILQOKDZK5FAMGXB2C' # your Foursquare ID
CLIENT_SECRET = '0D0TWC2MD2NDOHUUXRAYAS1DRQRAA2E14Z33QU01YADTE0O1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


# #intialise Foursquare API credentials
# CLIENT_ID = 'EKC0OWGJC1SY1AE1UHB4PUPH2JGARZTQK1U5C1USTUNA43JF' # your Foursquare ID
# CLIENT_SECRET = 'CHH0EPUEN2PH4WBDV4XHTDT5NWTUZ2SYVIQALUOZXWYNABRO' # your Foursquare Secret

### Get Venues

In [162]:
#Use the function to extract the category from the dataframe (because the column name could be either 'categories' or 'venue.categories')
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#define a function to return all the venues for a given neighborhood

#pass the neighborhood (nb), the latitude (la), the radius and the limit 
def get_venues(nb, la, lo, radius, limit):
    
    #print(nb)
    
    #form the request url and request only the items (which are the venues)
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, la, lo, VERSION, limit)
    items = requests.get(url).json()["response"]['groups'][0]['items']
    
    #include exception handling where Foursquare request fails for a particular Neighbourhood
    if items == []:
        print(" -- Foursquare request for ",nb,"returned no results --")
        return None
    else:
        # flatten JSON, filter for only wanted columns then use the get_category_type funtion to replace the category list with just the category.
        venues1 = json_normalize(items)
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        venues1 =venues1.loc[:, filtered_columns]
        venues1['venue.categories'] = venues1.apply(get_category_type, axis=1)

        #define a new dataframe with the Neighbourhood information
        venues2 = pd.DataFrame(columns=['Suburb','Suburb Latitude', 
                      'Suburb Longitude']) 

        #for each of the venues returned, add the venue dataframe to the (empty) neighbourhood dataframe and fill all rows in the the neighbourhood columns with the neighborhood name, the neighbourhood latitude and longitude
        i=0
        for row in venues1:
            venues2[row]=venues1[row]
            venues2['Suburb']=nb
            venues2['Suburb Latitude']=la
            venues2['Suburb Longitude']=lo
            i=i+1

        #rename the columns
        venues2.rename(index=str,columns={"venue.name":"Venue","venue.categories":"Venue Category","venue.location.lat":"Venue Latitude","venue.location.lng":"Venue Longitude"}, inplace=True)
    
    return venues2

### Call API

In [158]:
transport  = get_transport(TRANSPORTMODE, ORIGIN_X,ORIGIN_Y,DESTINATION_X,DESTINATION_Y,RETURN ,API_Key)
transport

https://router.hereapi.com/v8/routes?transportMode=pedestrian&origin=-37.8046041,144.9454422&destination=-37.8102306,144.9539645&return=polyline,summary&apikey=t9X9EF_AkG_qVZ8Q4dWSgyJuMhSklDP7i_gqX5EMunQ


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,type,polyline,departure.time,departure.place.type,departure.place.location.lat,departure.place.location.lng,departure.place.originalLocation.lat,departure.place.originalLocation.lng,arrival.time,arrival.place.type,arrival.place.location.lat,arrival.place.location.lng,arrival.place.originalLocation.lat,arrival.place.originalLocation.lng,summary.duration,summary.length,summary.baseDuration,transport.mode
id,,,,,,,,,,,,,,,,,,
efb71811-c925-4358-aae4-781073f8ddc6,pedestrian,BGtotjoCux4u0I5DyoBrEkwBvC0enB4STgFTsEnBsEnB4D...,2020-12-20T23:51:19+11:00,place,-37.804679,144.945431,-37.804604,144.945442,2020-12-21T00:08:45+11:00,place,-37.810234,144.953962,-37.810231,144.953965,1046,1039,1046,pedestrian


In [88]:
PolyLine = r['routes'][0]['sections'][0]['polyline']
Time = r['routes'][0]['sections'][0]['departure']['time']
location=r['routes'][0]['sections'][0]['departure']['place']['location']
originalLocation=r['routes'][0]['sections'][0]['departure']['place']['originalLocation']



In [130]:
r['routes'][0]

{'id': 'ef435536-ee29-4eb8-bad8-cd05e25b25c1',
 'sections': [{'id': 'd09890b6-9a98-412e-b2ed-ff1e47b89c07',
   'type': 'pedestrian',
   'departure': {'time': '2020-12-19T16:43:34+11:00',
    'place': {'type': 'place',
     'location': {'lat': -37.8046786, 'lng': 144.9454307},
     'originalLocation': {'lat': -37.8046042, 'lng': 144.9454422}}},
   'arrival': {'time': '2020-12-19T17:01:00+11:00',
    'place': {'type': 'place',
     'location': {'lat': -37.8102345, 'lng': 144.9539622},
     'originalLocation': {'lat': -37.8102306, 'lng': 144.9539645}}},
   'summary': {'duration': 1046, 'length': 1039, 'baseDuration': 1046},
   'polyline': 'BGtotjoCux4u0I5DyoBrEkwBvC0enB4STgFTsEnBsEnB4D7BsEjD8GjD0FrE8G_EwH_E8GjDgFjDgF3D8GnLoQjDsEjD4D_E0FrEgFzF8G7QoVrEoG3DgFvH4IvM4N3IsJzK0KrJsJ7LoL7QgP3IwHvCwC7G0FnLgKzFgFvM0KzFsE7GgFzF4D3IoGzFsErEkDrEwCzFkDnGkD_EoQ7BsOvCoQzF0K3IsJ3IA7GAvHoB_EwCjDkDjDgFnFwO',
   'transport': {'mode': 'pedestrian'}}]}

In [98]:
location

{'lat': -37.8046786, 'lng': 144.9454307}

In [99]:
pd.DataFrame.from_dict(r['routes'][0]['sections'][0]['departure']['place'])

,type,location,originalLocation
lat,place,-37.804679,-37.804604
lng,place,144.945431,144.945442


In [90]:
gpd

TypeError: 'module' object is not callable

In [64]:
r['routes'][0]['sections'][0].keys()

dict_keys(['id', 'type', 'departure', 'arrival', 'summary', 'polyline', 'transport'])

In [96]:
pd.DataFrame.from_dict(r['routes'][0]['sections'][0]['departure'])

,time,place
location,2020-12-19T16:43:34+11:00,"{'lat': -37.8046786, 'lng': 144.9454307}"
originalLocation,2020-12-19T16:43:34+11:00,"{'lat': -37.8046042, 'lng': 144.9454422}"
type,2020-12-19T16:43:34+11:00,place


In [77]:
location

{'lat': -37.8046786, 'lng': 144.9454307}

In [78]:
originalLocation

{'lat': -37.8046042, 'lng': 144.9454422}

In [63]:
r['routes'][0]['id']

'f902ae12-c7b5-4243-a47c-a22be2b1adfc'

In [66]:
PolyLine = r['routes'][0]['sections'][0]['polyline']

In [ ]:
r['routes'][0]['sections'][0]['departure']